In [ ]:
import numpy as np
import pandas as pd
import random
import time

In [ ]:
#This is one way of initilizating weights and bias
class Network(object):

    def __init__(self, sizes):
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]] #sizes[1:] skips the input layer (since it doesn't have biases) and includes all subsequent layers.
        self.weights = [np.random.randn(z, x) for x, z in zip(sizes[:-1], sizes[1:])] #zip(sizes[:-1], sizes[1:]) pairs each layer with the next one, creating tuples (x, z) where:
        # x is the number of neurons in the current layer.
        # z is the number of neurons in the next layer.
        print(f'sizes: {sizes}, biases: {self.biases}, weights: {self.weights}')

'''np.random.randn(y, 1):
For each layer y (number of neurons in that layer), this generates a column vector of shape (y, 1) with random values sampled from a standard normal distribution (mean = 0, standard deviation = 1).
Each value in this vector represents the initial bias for a neuron in that layer.'''

'''
List comprehension:
The list comprehension [np.random.randn(y, 1) for y in sizes[1:]] iterates over the number of neurons in each layer (excluding the input layer) and creates a bias vector for each layer.
'''
'''
Example:
If sizes = [784, 30, 10] (e.g., for an MNIST network with 784 input neurons, 30 hidden neurons, and 10 output neurons):

sizes[1:] = [30, 10]
self.biases will be a list containing:
A (30, 1) array for the biases of the hidden layer.
A (10, 1) array for the biases of the output layer.
'''

'''
Example:
If sizes = [784, 30, 10] (e.g., for an MNIST network with 784 input neurons, 30 hidden neurons, and 10 output neurons):

sizes[:-1] = [784, 30] (input and hidden layers).
sizes[1:] = [30, 10] (hidden and output layers).
zip(sizes[:-1], sizes[1:]) = [(784, 30), (30, 10)].
The resulting self.weights will be a list containing:

A (30, 784) matrix for the weights connecting the input layer to the hidden layer.
A (10, 30) matrix for the weights connecting the hidden layer to the output layer.
'''

The randn function in NumPy (np.random.randn) generates samples from a standard normal distribution (Gaussian distribution) with a mean of 0 and a standard deviation of 1.

In [11]:
#example
net = Network([2,3,1])
net

sizes: [2, 3, 1], biases: [array([[ 0.48450357],
       [ 0.570756  ],
       [-0.5221414 ]]), array([[0.31319019]])], weights: [array([[-1.54893345,  2.13447036],
       [-0.53880242, -0.44847638],
       [-0.14905836,  1.41607488]]), array([[-1.12878731, -0.07287262,  1.49177351]])]


In [ ]:
# Note that when the input z is a vector or Numpy array, Numpy automatically applies the function sigmoid elementwise, that is, in vectorized form.
def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))

In [ ]:
# We then add a feedforward method to the Network class, which, given an input a for the network, returns the corresponding output. All the method does is applies Equation (22) for each layer.
def feedforward(self, a):
    """Return the output of the network if "a" is input."""
    for b, w in zip(self.biases, self.weights):
        a = sigmoid(np.dot(w, a)+b)
    return a

In [ ]:
#Network will learn through stochastic gradient descent
"""Train the neural network using mini-batch stochastic gradient descent. The "training_data" is a list of tuples "(x, y)" representing the training inputs and the desired outputs.
If "test_data" is provided then the network will be evaluated against the test data after each epoch, and partial progress printed out.  This is useful for tracking progress, but slows 
things down substantially. The training_data is a list of tuples (x, y) representing the training inputs and corresponding desired outputs."""

def SGD(self, training_data, epochs, mini_batch_size, eta, test_data=None):
    if test_data: 
        n_test = len(test_data)
    n = len(training_data)
    for j in range(epochs):
        time1 = time.time()
        random.shuffle(training_data)
        mini_batches = [training_data[k:k+mini_batch_size] for k in range(0, n, mini_batch_size)]        
        for mini_batch in mini_batches:
            self.update_mini_batch(mini_batch, eta) #updates the network weights and biases according to a single iteration of gradient descent using training data in mini batch   
        time2 = time.time()
        if test_data:
            print("Epoch {0}: {1} / {2}, took {3:.2f} seconds".format(j, self.evaluate(test_data), n_test, time2-time1))
        else:
            print("Epoch {0} complete in {1:.2f} seconds".format(j, time2-time1))

In [ ]:
def update_mini_batch(self, mini_batch, eta):
    """Update the network's weights and biases by applying gradient descent using backpropagation to a single mini batch. The ``mini_batch`` is a list of tuples ``(x, y)``, and ``eta``
    is the learning rate."""
    nabla_b = [np.zeros(b.shape) for b in self.biases]
    nabla_w = [np.zeros(w.shape) for w in self.weights]
    for x, y in mini_batch:
        delta_nabla_b, delta_nabla_w = self.backprop(x, y) #This line invokes backpropagation algorithm which is a fast way of computing the gradient of the cost function.
        nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
        nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
    self.weights = [w-(eta/len(mini_batch))*nw for w, nw in zip(self.weights, nabla_w)]
    self.biases = [b-(eta/len(mini_batch))*nb for b, nb in zip(self.biases, nabla_b)]

In [ ]:
"""Derivative of the sigmoid function."""
def sigmoid_prime(z):
    return sigmoid(z)*(1-sigmoid(z))

In [ ]:
def backprop(self, x, y):
    """Return a tuple ``(nabla_b, nabla_w)`` representing the
    gradient for the cost function C_x.  ``nabla_b`` and
    ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
    to ``self.biases`` and ``self.weights``."""
    nabla_b = [np.zeros(b.shape) for b in self.biases]
    nabla_w = [np.zeros(w.shape) for w in self.weights]
    # feedforward
    activation = x
    activations = [x] # list to store all the activations, layer by layer
    zs = [] # list to store all the z vectors, layer by layer
    for b, w in zip(self.biases, self.weights):
        z = np.dot(w, activation)+b
        zs.append(z)
        activation = sigmoid(z)
        activations.append(activation)
    # backward pass
    delta = self.cost_derivative(activations[-1], y) * sigmoid_prime(zs[-1])
    nabla_b[-1] = delta
    nabla_w[-1] = np.dot(delta, activations[-2].transpose())
    # Note that the variable l in the loop below is used a little
    # differently to the notation in Chapter 2 of the book.  Here,
    # l = 1 means the last layer of neurons, l = 2 is the
    # second-last layer, and so on.  It's a renumbering of the
    # scheme in the book, used here to take advantage of the fact
    # that Python can use negative indices in lists.
    for l in range(2, self.num_layers):
        z = zs[-l]
        sp = sigmoid_prime(z)
        delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
        nabla_b[-l] = delta
        nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
    return (nabla_b, nabla_w)

In [ ]:
def evaluate(self, test_data):
    """Return the number of test inputs for which the neural
    network outputs the correct result. Note that the neural
    network's output is assumed to be the index of whichever
    neuron in the final layer has the highest activation."""
    test_results = [(np.argmax(self.feedforward(x)), y)
                    for (x, y) in test_data]
    return sum(int(x == y) for (x, y) in test_results)

In [ ]:
def cost_derivative(self, output_activations, y):
    """Return the vector of partial derivatives \partial C_x /
    \partial a for the output activations."""
    return (output_activations-y)